# Notebook for CME Futures Challenge

# Downloading historical data for indices (S&P, NASDAQ, DJIA)

Imports

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import re
from typing import List, Dict

Make get_data function for downloading from yf

In [ ]:
timeframe = '1000mo' # set timeframe

def get_data(tickers: List):
    data_dictionary = {}
    for ticker in tickers:
        data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
    return data_dictionary

# Getting Data

We need continuized data for returns information about futures since they roll over at expiration dates. I bought some continuous data for cheap, it's not back adjusted, which we could do - but I may try doing CV folds on each contract instead.

In [ ]:
es_df = pd.read_csv('ES.txt') # E S&P
nq_df = pd.read_csv('NQ.txt') # E Nasdaq
fv_df = pd.read_csv('FV.txt') # 5yr us treasury note contracts
ty_df = pd.read_csv('TY.txt') # 10yr US treasury bond contracts
us_df = pd.read_csv('US.txt') # 30yr US treasury bond contracts
gc_df = pd.read_csv('GC.txt') # Gold contracts
cl_df = pd.read_csv('CL.txt') # Crude oil contracts
jy_df = pd.read_csv('JY.txt') # JPY/USD contracts
bp_df = pd.read_csv('BP.txt') # BP/USD contracts
eu_df = pd.read_csv('EU.txt') # EURO/USD contracts


Clean these up a bit

In [ ]:
def rename_price_columns(data, item):
    # Rename columns
    data.rename(columns={'Open': f'{item}_open', 'High': f'{item}_high', 'Low': f'{item}_low', 'Close': f'{item}_close', 'Volume': f'{item}_volume'}, inplace=True)

In [ ]:
contract_data = {
    'ES': es_df,
    'NQ': nq_df,
    'FV': fv_df,
    'TY': ty_df,
    'US': us_df, 
    'GC': gc_df,
    'CL': cl_df,
    'JY': jy_df,
    'BP': bp_df,
    'EU': eu_df
}

for contract, data in contract_data.items():
    # Make sure date is datetime and set as index
    try:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index(data['Date'], inplace=True)
        data.drop(columns=['Date'], inplace=True)
    except:
        print('Index already fixed.')

    # Rename columns
    rename_price_columns(data, contract)


In [ ]:
contract_data['ES']

References for later

In [ ]:
indexes = ['S&P', 'NASDAQ', 'DJIA']
contracts = contract_data.keys() # These are the ones we are training on

Get yahoo finance data

In [ ]:
tickers = ['^GSPC', '^IXIC', '^DJI'] # S&P, NASDAQ, DJIA

# Download data
data_dictionary = get_data(tickers)

# Turn to df
s_p = pd.DataFrame(data_dictionary['^GSPC'])
nasdaq = pd.DataFrame(data_dictionary['^IXIC'])
djia = pd.DataFrame(data_dictionary['^DJI'])

Fix structure

In [ ]:
# Flatten df so ticker doesn't span the top
s_p = s_p.droplevel(1, axis=1)
nasdaq = nasdaq.droplevel(1, axis=1)
djia = djia.droplevel(1, axis=1)

# Drop unnecessary columns
#s_p.drop(columns=['High', 'Low', 'Open'], inplace=True)
#nasdaq.drop(columns=['High', 'Low', 'Open'], inplace=True)
#djia.drop(columns=['High', 'Low', 'Open'], inplace=True)

index_data = {
    'S&P': s_p,
    'NASDAQ': nasdaq,
    'DJIA': djia,
}

for index, data in index_data.items():
    # Rename columns
    rename_price_columns(data, index)

# Downloading historical data for our factor model

We are going to model the index as a geometric brownian motion, with the mu factor being a linear regression model with numerous inputs.  

## Factor considerations:  
### <u>Term structure</u>
###### Term spread (10Y-3M)

### <u>Credit conditions</u>
###### IG spread (BAA-AAA)

### <u>Valuation</u>
###### Forward E/P - real 10Y
###### Dividend yield

### <u>Economic</u>
###### Fed funds
###### Inflation (CPI)
###### DXY change (dollar index)  

### Some of these we can get from yahoo finance:  

In [ ]:
tickers = [
    # Term structure
    '^TNX', # 10yr CBOE
    '^IRX', # 3m bill (on discount basis, need to convert to yield)

    # Economic
    'DX-Y.NYB', # Dollar index

    # Volatility
    '^VIX',

    # Gold could be a measure of macro activity?
    "GLD", # Gold ETF

    # Other metals
    "XME", # SPDR Metals and mining ETF

    # Credit
    'FLRN', # Bloomberg IG Credit ETF
    'JNK', # Bloomberg HY Credit ETF

]

data_dictionary = get_data(tickers)

for ticker, data in data_dictionary.items():
    data_dictionary[ticker] = pd.DataFrame(data_dictionary[ticker])
    data_dictionary[ticker] = data_dictionary[ticker].droplevel(1, axis=1)
    rename_price_columns(data_dictionary[ticker], ticker)

ten_yr = data_dictionary['^TNX']
three_m = data_dictionary['^IRX']
dollar_index = data_dictionary['DX-Y.NYB']
vix = data_dictionary['^VIX']

# Need to drop volume column here
ten_yr.drop(columns=['^TNX_volume'], inplace=True)
three_m.drop(columns=['^IRX_volume'], inplace=True)
dollar_index.drop(columns=['DX-Y.NYB_volume'], inplace=True)
vix.drop(columns=['^VIX_volume'], inplace=True)


#gold = data_dictionary['GLD']
#metals = data_dictionary['XME']
#igb = pd.DataFrame(data_dictionary['FLRN'])
#hyb = data_dictionary['JNK']

In [ ]:
yf_macros = [vix, ten_yr, three_m, dollar_index]

We should get dividend yield too

In [ ]:
div_data = {}
etfs = ['SPY', 'QQQ', 'DIA']

for etf in etfs:
    ticker = yf.Ticker(etf)
    div = ticker.dividends
    price = ticker.history(timeframe)['Close']

    # Calculate dividend yield
    div_12m = div.rolling(window='365D', min_periods=1).sum()
    div_12m = div_12m.reindex(price.index, method='ffill')
    div_yield = div_12m / price
    div_data[etf] = div_yield

Fix index for all 3 and rename columns

In [ ]:
div_data['SPY'].index = pd.to_datetime(div_data['SPY'].index).normalize().tz_localize(None) # Normalize puts date in format we want
div_data['QQQ'].index = pd.to_datetime(div_data['QQQ'].index).normalize().tz_localize(None) # Localize (none) makes sure it doesn't add our timezone
div_data['DIA'].index = pd.to_datetime(div_data['DIA'].index).normalize().tz_localize(None)

div_data['SPY'].name = 'S&P_div'
div_data['QQQ'].name = 'NASDAQ_div'
div_data['DIA'].name = 'DJIA_div'

### pandas_datareader lets us download fred data

In [ ]:
from pandas_datareader import data as pdr
from datetime import datetime

In [ ]:
start = datetime(1990,1,1) # Start date for download

# Macroeconomic data
gdp = pdr.DataReader("GDP", "fred", start)
cpi = pdr.DataReader("CPIAUCSL", "fred", start)
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start)
consumer_sentiment = pdr.DataReader("UMCSENT", "fred", start)
inflation_expectation = pdr.DataReader("MICH", "fred", start)

# For some reason this download doesn't have the most recent fed funds rate
fedfunds = pd.concat([fedfunds['FEDFUNDS'], pd.Series([4.08], index=[datetime(2025,9,17)])])

# Credit risk data
ig_spread = pdr.DataReader("BAMLC0A4CBBB", "fred", start)   # BofA BBB corp minus Treasuries
#hy_spread = pdr.DataReader("BAMLH0A0HYM2", "fred", start)   # BofA US High Yield spread
#baa_spread = pdr.DataReader("BAA10Y", "fred", start)        # Moody’s Baa – 10Y Treasury

In [ ]:
fred_data = [gdp, cpi, fedfunds, ig_spread, consumer_sentiment, inflation_expectation]

In [ ]:
# Last business day <= today
last_bday = pd.bdate_range(end=pd.Timestamp.today().normalize().tz_localize(None), periods=1)[0]

for i, df in enumerate(fred_data):
    s = df.squeeze() # make it a Series
    # Build a business-day index from the series start to last_bday
    bidx = pd.bdate_range(start=s.index.min(), end=last_bday)
    # Reindex to business days and forward-fill
    s = s.reindex(bidx, method='ffill')
    # Write back as a 1-col DataFrame with a proper name
    name = s.name if s.name else f"series_{i}"
    fred_data[i] = s.to_frame(name)

Let's build a master dataframe

In [ ]:
pd.set_option('display.max_columns', None)
data = s_p.join([nasdaq, djia])
data = data.join(contract for contract in contract_data.values())
data = data.join(macro for macro in yf_macros)
data = data.join(div for div in div_data.values())
data = data.join([fd for fd in fred_data])

Rename some columns

In [ ]:
data.rename(columns={'CPIAUCSL': 'CPI', 'BAMLC0A4CBBB': 'credit_spread', 'UMCSENT': 'consumer_sentiment', 'MICH': 'inflation_expectation', 'series_2': 'fed_funds'}, inplace=True)
data[data.index.year > 2000]

# Linear regression model

### Preprocessing Data

Let's check for NaNs (there's a ton)

In [ ]:
data.isna().sum()

Check data types

In [ ]:
data.dtypes

Fill forward for any gaps

In [ ]:
data = data.replace(0, np.nan)
data = data.fillna(method='ffill')

Drop others

In [ ]:
data = data.dropna()
data

# Feature Engineering

We need to be careful to not include things such as raw moving averages that will leak volatility information into our drift prediction  

Forward adjusting futures

In [ ]:
roll_rules = { # Rollover occurances: (months, days)
    'ES': (0,5),
    'NQ': (0,5),
    'FV': (1,1),
    'TY': (1,1),
    'US': (1,1),
    'GC': (1,2),
    'CL': (0,3),
    'JY': (0,2),
    'BP': (0,2),
    'EU': (0,2)
}

In [ ]:
IMM_QTR = {3, 6, 9, 12}
GOLD_MONTHS = {2, 4, 6, 8, 10, 12}

def _third_fridays_quarterly(start, end):
    d = pd.date_range(start, end, freq="WOM-3FRI")
    return d[d.month.isin(IMM_QTR)]

def _bmonth_end_quarterly(start, end):
    # Last business day of IMM months
    all_month_ends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    return all_month_ends[all_month_ends.month.isin(IMM_QTR)]

def _third_wednesdays_quarterly(start, end):
    # 3rd Wednesday of IMM months
    weds = pd.date_range(start, end, freq="WOM-3WED")
    return weds[weds.month.isin(IMM_QTR)]

def _gold_expiries(start, end):
    # 3rd-last business day of bi-monthly GC months (anchor only)
    mends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    mends = mends[mends.month.isin(GOLD_MONTHS)]
    # 3rd last BD in that month = BM - 2 BDays
    return (mends - pd.offsets.BusinessDay(2))

def _cl_expiries(start, end):
    # Crude "expiration" day (last trade day): 3 business days before the 25th of the PRIOR month
    # We'll generate it for EVERY contract month in range.
    cl_expiries = []
    # iterate month starts to month ends
    month_starts = pd.date_range(start.normalize(), end.normalize(), freq="MS")
    for ms in month_starts:
        # prior month 25th
        prior_month_25th = (ms - pd.offsets.MonthBegin(1)) + pd.offsets.Day(24)  # 25th of prior month
        # adjust 25th to previous business day if needed, then subtract 3 business days
        # (approximation: we ignore exchange holidays and use business days = weekdays)
        # Find the previous business day <= prior_month_25th
        prev_bd = prior_month_25th
        # step back to weekday if falls on weekend
        while prev_bd.weekday() >= 5:
            prev_bd -= pd.Timedelta(days=1)
        ltd = prev_bd - pd.offsets.BusinessDay(3)
        cl_expiries.append(ltd.normalize())
    return pd.DatetimeIndex(sorted(set(cl_expiries)))

def _expiry_calendar(symbol, start, end):
    s = symbol.upper()
    if s in {"ES", "NQ"}:
        return _third_fridays_quarterly(start, end)
    if s in {"FV", "TY", "US"}:
        return _bmonth_end_quarterly(start, end)
    if s == "GC":
        return _gold_expiries(start, end)
    if s == "CL":
        return _cl_expiries(start, end)
    if s in {"JY", "BP", "EU"}:
        return _third_wednesdays_quarterly(start, end)
    # Fallback: quarterly 3rd Fridays (reasonable default)
    return _third_fridays_quarterly(start, end)

# -----------------------
# main: compute rollover flags, episodes, and DTR
# -----------------------
def get_rollover_data(data: pd.DataFrame, item: str, roll_rules: dict):
    months_off, days_off = roll_rules[item]  # (months, days)

    # We add a buffer so we can see the next expiry at the end of the dataset
    start = pd.Timestamp(data.index.min()).normalize()
    end = (pd.Timestamp(data.index.max()).normalize() + pd.DateOffset(months=4))

    expiries = _expiry_calendar(item, start, end)

    # Compute roll dates per rule
    if months_off == 0:
        roll_dates = expiries - pd.offsets.BusinessDay(days_off)
    else:
        # For each expiry, go back N months to that month's LAST business day, then step back 'days_off'
        anchor_month_ends = (expiries - pd.offsets.MonthBegin(months_off))  # first of prior month(s)
        # move to that month's last business day
        anchor_bm_end = pd.to_datetime(anchor_month_ends).to_period('M').to_timestamp('M')
        # ensure it's a business day (BM gives last business day; if not, use BMonthEnd)
        anchor_bm_end = pd.DatetimeIndex(anchor_bm_end)  # calendar month-end
        # Convert to last BUSINESS day of that month
        anchor_last_bd = pd.DatetimeIndex([pd.Timestamp(d).to_period('M').to_timestamp('M') for d in anchor_bm_end])
        # Replace with BusinessMonthEnd for precision
        anchor_last_bd = pd.date_range(anchor_last_bd.min(), anchor_last_bd.max(), freq="BM").intersection(anchor_last_bd)
        # The above intersection collapses; safer approach:
        # Recompute last BD robustly per expiry:
        last_bds = []
        for e in expiries:
            # month = e - months_off
            mstart = (e - pd.offsets.MonthBegin(months_off))
            # last business day of that month:
            lbd = (mstart + pd.offsets.BMonthEnd(0))
            last_bds.append(lbd)
        anchor_last_bd = pd.DatetimeIndex(last_bds)

        roll_dates = anchor_last_bd - pd.offsets.BusinessDay(days_off)

    roll_dates = pd.DatetimeIndex(sorted(set(pd.to_datetime(roll_dates))))
    # Keep only roll dates that appear in our data index (typical)
    roll_dates = roll_dates.intersection(pd.DatetimeIndex(data.index))

    # Flags and episodes
    rflag = f"{item}_Rollover"
    eph = f"{item}_Episode"
    data[rflag] = 0
    if len(roll_dates):
        data.loc[roll_dates, rflag] = 1
    data[eph] = data[rflag].cumsum()

    # Days until next rollover (trading days)
    # Forward-fill the next roll date and count business days
    next_roll = (
        pd.Series(roll_dates, index=roll_dates)
        .reindex(data.index, method="bfill")
        .to_numpy()
    )
    days = np.full(len(data), np.nan)
    mask = ~pd.isna(next_roll)
    # convert to daily (no time) for busday_count
    left = data.index.to_numpy(dtype="datetime64[D]")
    right = pd.to_datetime(next_roll).to_numpy(dtype="datetime64[D]")
    days[mask] = np.busday_count(left[mask], right[mask])  # excludes the right endpoint
    data[f"{item}_Days_Until_Rollover"] = days

# -----------------------
# forward adjustment on roll days
# -----------------------
def rollover_forward_adjustment(data: pd.DataFrame, contract: str):
    rflag = f"{contract}_Rollover"
    close = f"{contract}_close"
    # Jump only on the roll day = today's close minus yesterday's close
    jump = data[rflag].fillna(0).astype(int) * (data[close] - data[close].shift(1).fillna(0.0))
    cum_adj = jump.cumsum()
    for col in ["open", "high", "low", "close"]:
        c = f"{contract}_{col}"
        if c in data.columns:
            data[c] = data[c] - cum_adj

In [ ]:
for contract in contracts:
    # Set info about expirations
    get_rollover_data(data, contract, roll_rules)

    # Forward adjust rollover
    rollover_forward_adjustment(data, contract)
data.dropna(inplace=True)

Function definitions to help out

In [ ]:
def rolling_mean(data, window):
    return data.rolling(window, min_periods=window).mean()

### Features

Technical Indicators

In [ ]:
def add_technical_indicators(data, items, cfg=None, dropna=True):
    if cfg is None:
        cfg = {
            "rsi_n": 14,
            "macd_fast": 12, "macd_slow": 26, "macd_signal": 9,
            "bb_n": 20, "bb_k": 2.0,
            "atr_n": 14
        }
    out = data.copy()

    for item in items:
        o = out[f"{item}_open"].astype(float)
        h = out[f"{item}_high"].astype(float)
        l = out[f"{item}_low"].astype(float)
        c = out[f"{item}_close"].astype(float)
        v = out[f"{item}_volume"].astype(float)

        # --- 1) RSI (Wilder smoothing) ---
        n = cfg["rsi_n"]
        delta = c.diff()
        gain = delta.clip(lower=0.0)
        loss = (-delta).clip(lower=0.0)
        avg_gain = gain.ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        avg_loss = loss.ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        rs = avg_gain / (avg_loss.replace(0, np.nan))
        rsi = 100 - (100 / (1 + rs))
        out[f"{item}_rsi_{n}"] = rsi

        # --- 2) MACD (EMA fast/slow, signal, hist) ---
        ema_fast = c.ewm(span=cfg["macd_fast"], adjust=False).mean()
        ema_slow = c.ewm(span=cfg["macd_slow"], adjust=False).mean()
        macd = ema_fast - ema_slow
        macd_signal = macd.ewm(span=cfg["macd_signal"], adjust=False).mean()
        macd_hist = macd - macd_signal
        out[f"{item}_macd"] = macd
        out[f"{item}_macd_signal"] = macd_signal
        out[f"{item}_macd_hist"] = macd_hist

        # --- 3) Bollinger Bands (+ %B, Bandwidth) ---
        bb_n = cfg["bb_n"]; bb_k = cfg["bb_k"]
        mavg = c.rolling(bb_n, min_periods=bb_n).mean()
        mstd = c.rolling(bb_n, min_periods=bb_n).std(ddof=0)
        upper = mavg + bb_k * mstd
        lower = mavg - bb_k * mstd
        pct_b = (c - lower) / (upper - lower)
        bandwidth = (upper - lower) / mavg
        out[f"{item}_bb_mean_{bb_n}"] = mavg
        out[f"{item}_bb_upper_{bb_n}_{int(bb_k)}"] = upper
        out[f"{item}_bb_lower_{bb_n}_{int(bb_k)}"] = lower
        out[f"{item}_bb_percent_b"] = pct_b
        out[f"{item}_bb_bandwidth"] = bandwidth

        # --- 4) ATR (Wilder, uses True Range) ---
        prev_c = c.shift(1)
        tr = np.maximum(h - l, np.maximum((h - prev_c).abs(), (l - prev_c).abs()))
        atr = tr.ewm(alpha=1/cfg["atr_n"], adjust=False, min_periods=cfg["atr_n"]).mean()
        out[f"{item}_atr_{cfg['atr_n']}"] = atr

        # --- 5) OBV (On-Balance Volume) ---
        sign = np.sign(c.diff().fillna(0.0))
        obv = (sign * v).fillna(0.0).cumsum()
        out[f"{item}_obv"] = obv

    if dropna:
        out = out.dropna()

    return out

In [ ]:
data = add_technical_indicators(data, contracts)

In [ ]:
def get_log_price(data, items):
    data = data.copy()
    
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    for item in items:
        for key in ohlcv:
            data[f'{item}_log_{key}'] = np.log(data[f'{item}_{key}'])
            data.drop(columns=[f'{item}_{key}'], inplace=True) # Drop originals

        data.dropna(inplace=True)
    return data

In [ ]:
def get_log_returns(data, items):
    data = data.copy()
    
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    for item in items:
        for key in ohlcv:
            data[f'{item}_log_{key}_ret'] = data[f'{item}_log_{key}'].diff()

        data[f'{item}_log_open_close_ret'] = data[f'{item}_log_close'] - data[f'{item}_log_open']

    data.dropna(inplace=True)
    return data

In [ ]:
# Get log transforms of price data
log_price_keys = indexes + list(contract_data.keys()) #+ ['GLD', 'XME', 'JNK']

# Log prices
data = get_log_price(data, log_price_keys)

# Log returns
data = get_log_returns(data, log_price_keys)

# Log for other price data (GDP)
data['log_GDP'] = np.log(data['GDP'])
data.drop(columns=['GDP'], inplace=True)
data.dropna(inplace=True)

Price/Volume Features

In [ ]:
def get_price_volume_features(data, items):
    data = data.copy()
    
    for item in items:
        # Price-based
        data[f'{item}_mom_1w'] = data[f'{item}_log_close'].diff(5) # Total price change / momentum indicator
        data[f'{item}_mom_3m'] = data[f'{item}_log_close'].diff(63)
        data[f'{item}_1m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 21)
        data[f'{item}_3m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 63)
        data[f'{item}_6m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 126)
        data[f'{item}_trend_speed_price'] = data[f'{item}_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
        data[f'{item}_trend_dist_price'] = data[f'{item}_log_close'] - data[f'{item}_3m_rolling_price']

        # Volume-based (essentially the same as price for now)
        data[f'{item}_vlm_1w'] = data[f'{item}_log_volume'].diff(5) # Total volume change / momentum indicator
        data[f'{item}_vlm_1m'] = data[f'{item}_log_volume'].diff(21)
        data[f'{item}_vlm_3m'] = data[f'{item}_log_volume'].diff(63)
        data[f'{item}_3m_rolling_volume'] = rolling_mean(data[f'{item}_log_volume'], 63)
        data[f'{item}_trend_speed_volume'] = data[f'{item}_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
        data[f'{item}_trend_dist_volume'] = data[f'{item}_log_volume'] - data[f'{item}_3m_rolling_volume']

    return data

In [ ]:
data = get_price_volume_features(data, log_price_keys)
data.dropna(inplace=True)

Macro features

In [ ]:
data['log_GDP_ret'] = data['log_GDP'].diff()
data['log_GDP_ret_1y'] = data['log_GDP'].diff(252)
data['inflation_expectation_ret'] = data['inflation_expectation'].pct_change()
data['consumer_sentiment_ret'] = data['consumer_sentiment'].pct_change()
data['cpi_ret'] = data['CPI'].pct_change()
data.dropna(inplace=True)

Spreads

In [ ]:
data['Spread_NQ_ES_ret'] = data['ES_log_close_ret'] - data['NQ_log_close_ret']
data['Spread_NQ_ES_volume'] = data['ES_3m_rolling_volume'] - data['NQ_3m_rolling_volume']
data['Spread_ES_GC_ret'] = data['ES_log_close_ret'] - data['GC_log_close_ret']
data['Spread_ES_GC_volume'] = data['ES_3m_rolling_volume'] - data['GC_3m_rolling_volume']
data['Spread_FV_US_ret'] = data['FV_log_close_ret'] - data['US_log_close_ret']
data['Spread_FV_US_volume'] = data['FV_3m_rolling_volume'] - data['US_3m_rolling_volume']
data['Spread_TY_US_ret'] = data['TY_log_close_ret'] - data['US_log_close_ret']
data['Spread_TY_US_volume'] = data['TY_3m_rolling_volume'] - data['US_3m_rolling_volume']
data.dropna(inplace=True)

Calendar dummies

In [ ]:
try:
    month_dummies = pd.get_dummies(data.index.month, prefix="month", dtype=float)
    month_dummies.set_index(data.index, inplace=True)
    day_dummies = pd.get_dummies(data.index.dayofweek, prefix="weekday", dtype=float)
    day_dummies.set_index(data.index, inplace=True)
    presidency_year_dummies = pd.get_dummies(data.index.year % 4, prefix='pres_year', dtype='float')
    presidency_year_dummies.set_index(data.index, inplace=True)
    
    data = data.join([month_dummies, day_dummies, presidency_year_dummies])
except:
    print('Error: probably already created dummies.')

Volatility / Price Features (From ChatGPT)

In [ ]:
daily_vix_vol = data['^VIX_close'] / (100 * np.sqrt(252))
data['vix_implied_var'] = np.square(daily_vix_vol)
    # Measures the percentage change in the ^VIX over 1 week and 1 month
data['vix_mom_1w'] = data['^VIX_close'].pct_change(periods=5)
data['vix_mom_1m'] = data['^VIX_close'].pct_change(periods=21)

data[f'vix_trend_1m'] = rolling_mean(data[f'^VIX_close'], 21)
data[f'vix_trend_3m'] = rolling_mean(data[f'^VIX_close'], 63)
data[f'vix_trend_6m'] = rolling_mean(data[f'^VIX_close'], 126)

# Calculate how far the current VIX is from its trend (as a percentage)
data['vix_trend_dist'] = (data['^VIX_close'] - data['vix_trend_3m']) / data['vix_trend_3m']

In [ ]:
EPS = 1e-12

# If you already have a rolling_mean helper, keep it. Otherwise:
def rolling_mean(s, w):
    return s.rolling(w, min_periods=max(2, int(w*0.6))).mean()

def ewma_vol(r, lam=0.94):
    # EWMA variance per RiskMetrics: sigma_t^2 = (1-lam)*r_{t-1}^2 + lam*sigma_{t-1}^2
    # Use pandas ewm for convenience
    return r.pow(2).ewm(alpha=(1-lam), adjust=False).mean().clip(lower=0)

def rolling_autocorr(x, lag=1, window=63):
    # Rolling autocorrelation of x at a given lag
    # For stability, require at least ~60% of window
    minp = max(10, int(window*0.6))
    x0 = x
    x1 = x.shift(lag)
    return x0.rolling(window, min_periods=minp).corr(x1)

def realized_quarticity(r, window=63):
    # 3-month robust quarticity proxy (if daily): sum r^4 * (n / 3) approximation
    # Here we simply provide rolling sum of r^4; scaling optional depending on use
    minp = max(10, int(window*0.6))
    return (r.pow(4)).rolling(window, min_periods=minp).sum()

def build_vol_features(data, prefix, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94, target='close'):
    """
    Expects:
      data[f'{prefix}_log_price'] (daily log price)
      data[f'{prefix}_log_volume'] (daily log volume)
    Produces a suite of volatility-centric features for that prefix.
    """
    
    lp = data[f"{prefix}_log_{target}"]
    lv = data.get(f"{prefix}_log_volume", None)

    # Daily log return
    r = lp.diff()  # already log-price, so diff = log-return

    # --- Realized volatility proxies ---
    data[f"{prefix}_rv_1m"]  = r.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)          # variance
    data[f"{prefix}_rv_3m"]  = r.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_rv_1y"]  = r.rolling(yr_w,  min_periods=int(yr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_absrv_1m"] = r.abs().rolling(day_w, min_periods=int(day_w*0.6)).mean()               # mean |r|
    data[f"{prefix}_absrv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).mean()

    # EWMA volatility (RiskMetrics-style)
    data[f"{prefix}_ewma_var"] = ewma_vol(r, lam=ewma_lambda)
    data[f"{prefix}_ewma_vol"] = np.sqrt(data[f"{prefix}_ewma_var"])

    # Volatility-of-volatility (how fast vol is changing)
    data[f"{prefix}_vol_speed_1w"] = data[f"{prefix}_rv_3m"].diff(5)                                      # weekly change in 3m var
    data[f"{prefix}_vol_mom_1m"]   = data[f"{prefix}_rv_3m"] - data[f"{prefix}_rv_1m"]                    # 3m vs 1m
    data[f"{prefix}_vol_mom_1y"]   = data[f"{prefix}_rv_1y"] - data[f"{prefix}_rv_3m"]

    # Volatility clustering proxies
    data[f"{prefix}_acf_sqret_lag1_3m"] = rolling_autocorr(r.pow(2), lag=1, window=qtr_w)
    data[f"{prefix}_acf_absret_lag1_3m"] = rolling_autocorr(r.abs(), lag=1, window=qtr_w)

    # Leverage effect proxy (contemporaneous corr between return and next day's vol)
    # Negative returns often precede higher vol; we proxy with corr(r_t, |r|_{t+1})
    data[f"{prefix}_lev_proxy_3m"] = r.shift(1).rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(r.abs()).shift(1)

    # Quarticity (heavy tails proxy)
    data[f"{prefix}_quarticity_3m"] = realized_quarticity(r, window=qtr_w)

    # Ratio features (normalized vol levels)
    data[f"{prefix}_vol_ratio_1m_3m"] = (data[f"{prefix}_rv_1m"] / (data[f"{prefix}_rv_3m"] + EPS))
    data[f"{prefix}_vol_ratio_3m_1y"] = (data[f"{prefix}_rv_3m"] / (data[f"{prefix}_rv_1y"] + EPS))
    data[f"{prefix}_ewma_over_3m"]    = (data[f"{prefix}_ewma_var"] / (data[f"{prefix}_rv_3m"] + EPS))

    # Price–volatility relation: distance from trend as a stress proxy
    data[f"{prefix}_price_trend_3m"]  = rolling_mean(lp, qtr_w)
    data[f"{prefix}_price_trend_dist"] = lp - data[f"{prefix}_price_trend_3m"]
    # Volatility when far below trend often spikes; include interaction
    data[f"{prefix}_vol_x_trend_dist"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_price_trend_dist"]

    # Volume–volatility links (if volume available)
    if lv is not None:
        dv = lv.diff()  # log-volume change
        data[f"{prefix}_vlm_var_1m"] = dv.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)
        data[f"{prefix}_vlm_var_3m"] = dv.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
        # Corr between |r| and volume changes (vol–volume clustering)
        data[f"{prefix}_corr_absr_dlv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(dv)
        # Volume surprise proxy: current vs 3m trend
        data[f"{prefix}_vlm_trend_3m"] = rolling_mean(lv, qtr_w)
        data[f"{prefix}_vlm_trend_dist"] = lv - data[f"{prefix}_vlm_trend_3m"]
        # Vol reacts to volume surprises
        data[f"{prefix}_vol_x_vlm_surprise"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_vlm_trend_dist"]

    # 2. Calculate the spread against a realized variance feature you already have
    # We'll use a 1-month realized variance (rv_1m) for the S&P 500 as an example.
    # Ensure you have 'S&P_rv_1m' or a similar column from your previous code.
    data['{prefix}_vrp_spread'] = data['vix_implied_var'] - data[f'{prefix}_rv_1m']



    return data

# ---- Apply to all price series ---- #### both close and close_ret
for item in log_price_keys:
    data = build_vol_features(data, index, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94)
    data = build_vol_features(data, item, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94, target='close_ret')

# ---- Cross-index spillover features (optional but useful) ----
# Differences/spreads in contemporaneous vol across indices capture contagion/regime moves
data["SPX_minus_NDX_vol_1m"] = data["S&P_rv_1m"] - data["NASDAQ_rv_1m"]
data["SPX_minus_DJIA_vol_1m"] = data["S&P_rv_1m"] - data["DJIA_rv_1m"]
data["NDX_minus_DJIA_vol_1m"] = data["NASDAQ_rv_1m"] - data["DJIA_rv_1m"]
data['ES_minus_NQ_vol_1m'] = data["ES_rv_1m"] - data["NQ_rv_1m"]

In [ ]:
data.dropna(inplace=True)

### Split data

Training/testing 80/20 split

In [ ]:
import math

### Set target

In [ ]:
linear_model_target = 'next_ret'
drift = data.copy()

In [ ]:
for contract in contracts:
    drift[f'{contract}_{linear_model_target}'] = drift[f'{contract}_log_close_ret'].shift(-1)
drift.dropna(inplace=True)

In [ ]:
def split_data(data, split=0.8, embargo=2):
    cutoff = math.floor(len(data)*split)
    training_data = data.iloc[:(cutoff - embargo)]
    testing_data = data.iloc[(cutoff+embargo):]
    return training_data.copy(), testing_data.copy()

In [ ]:
drift_training_data, drift_testing_data = split_data(drift)

### Determine feature importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance

In [ ]:
def set_targets_features(target, training_data, contracts=contracts, additional_exclusions=[], additional_unscalables=[]):
    # Make sure we only fit on training_data and explanatory variables
    targets = [f'{contract}_{target}' for contract in contracts]
    month_dummies = [f'month_{month}' for month in range(1,13)]
    day_dummies = [f'weekday_{day}' for day in range(5)]
    presidency_year_dummies = day_dummies = [f'pres_year_{year}' for year in range(4)]
    rollover_dummy = [f'{contract}_Rollover' for contract in contracts]
    dummies = month_dummies + day_dummies + presidency_year_dummies + rollover_dummy
    additional_exclusions.extend([f'{contract}_Episode'])


    columns_to_exclude = [] + additional_exclusions

    unscalable_features = dummies + additional_unscalables
    scalable_features = [column for column in training_data.columns if column not in targets and column not in unscalable_features and column not in columns_to_exclude]
    all_features = unscalable_features + scalable_features

    return targets, all_features, unscalable_features, scalable_features

In [ ]:
drift_targets, drift_all_features, drift_unscale_features, drift_scale_features = set_targets_features(linear_model_target, drift_training_data)

Let's try using exp weight decay

In [ ]:
def determine_feature_importance(training_data, testing_data, features, target, contracts=contracts, mode='permutation', permutation_threshold = 0, selector_threshold='median'):
    pruned_features = {}

    for contract in contracts:
        y_train = training_data[f'{contract}_{target}']
        y_test = testing_data[f'{contract}_{target}']
        X_train = training_data[features]
        X_test = testing_data[features]

        # Fit random forest
        rf = RandomForestRegressor(
            n_estimators = 500,
            max_depth=None,
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)

        if mode == 'permutation':
            result = permutation_importance(
                rf, X_test, y_test, # Make sure on test data
                n_repeats = 30,
                random_state=42,
                n_jobs=-1,
            )

            importances = pd.Series(result.importances_mean, index=X_train.columns).sort_values(ascending=False)
            selected_features = importances[importances > permutation_threshold].index.tolist()

            pruned_features[contract] = {
                'train_reduced': X_train[selected_features],
                'test_reduced': X_test[selected_features],
                'selected_features': selected_features
            }


        if mode == 'random_forest':
            selector = SelectFromModel(rf, threshold=selector_threshold, prefit=True)
            selected_features = training_data[features].columns[selector.get_support()]

            pruned_features[contract] = {
                'train_reduced': selector.transform(X_train),
                'test_reduced': selector.transform(X_test),
                'selected_features': selected_features
            }

    return pruned_features

Pickling for saving compute expensive dicts

In [ ]:
import os, pickle, gzip, bz2, lzma
from pathlib import Path
from typing import Callable, Any, Tuple

In [ ]:
def _opener(path: Path, mode: str):
    ext = path.suffix.lower()
    if ext in ('.pkl', '.pickle'):
        return path.open(mode)
    if ext == '.gz':
        return gzip.open(path, mode)
    if ext == '.bz2':
        return bz2.open(path, mode)
    if ext in ('.xz', '.lzma'):
        return lzma.open(path, mode)
    raise ValueError(f"Unsupported extension: {ext}")

def get_or_build_results(
    path: str,
    build_fn: Callable[..., dict],
    *args, **kwargs
) -> Tuple[dict, str]:

    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)

    # 1) Try load
    try:
        with _opener(p, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:

        try:
            # 2) Build
            results = build_fn(*args, **kwargs)

            # 3) Save
            with _opener(p, 'wb') as f:
                pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print(f'Exception occured: {e}')

    return results

In [ ]:
# Random forest for now because it's faster
pruned_features = get_or_build_results('model_data/drift_features_rf.pkl.gz', determine_feature_importance, drift_training_data, drift_testing_data, drift_all_features, 'next_ret', mode='random_forest')

for contract in contracts:
    print(pruned_features[contract]['selected_features'])

### Normalize inputs

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [ ]:
def scale_features(training_data, testing_data, features):
    scaler = RobustScaler()
    scaler.fit(training_data[features]) # Fitting on training data

    train_scaled = training_data.copy()
    test_scaled = testing_data.copy()

    # Scale
    train_scaled[features] = scaler.transform(training_data[features])
    test_scaled[features] = scaler.transform(testing_data[features])

    # Save info on standardization for later
    #scaler_mu = pd.Series(scaler.mean_, index=features)
    #scaler_std = pd.Series(scaler.scale_, index=features)
    return train_scaled, test_scaled #, scaler_mu, scaler_std

In [ ]:
drift_train_scaled, drift_test_scaled = scale_features(drift_training_data, drift_testing_data, drift_scale_features)

### Linear Regression

In [ ]:
import warnings
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.cross_decomposition import PLSRegression
from mlxtend.evaluate import GroupTimeSeriesSplit

We are going to test with multiple different linear models to account for collinearity

In [ ]:
# Choose which models we want to use
linear_models = ['ridge', 'lasso', 'enet', 'pca'] 

In [ ]:
# Function to get print results from the models
def eval_and_report(y_true, y_pred, model_name):
    print(f"{model_name:18s} | R^2: {r2_score(y_true, y_pred):.4f} | RMSE: {root_mean_squared_error(y_true, y_pred):.6f}")

Training function

In [ ]:
def train_models(train_scaled, test_scaled, target, contracts=contracts, pruned_features=pruned_features, models=linear_models):
    # Suppress all convergence warnings
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    results = {}

    for contract in contracts:
        results[contract] = {}
        features = pruned_features[contract]['selected_features']

        print(f"\n=== Target: {f'{contract}_{target}'} ===")
        X_train = train_scaled[features].copy()
        y_train = train_scaled[f'{contract}_{target}'].copy()
        X_test = test_scaled[features].copy()
        y_test = test_scaled[f'{contract}_{target}'].copy()

        # Create time series splits that isolate each contract
        ep_train = train_scaled[f'{contract}_Episode']
        gts = GroupTimeSeriesSplit(n_splits = ep_train.nunique()-2, test_size=1, window_type='expanding', gap_size=1)
        
        tscv = list(gts.split(
            X = X_train,
            y = y_train,
            groups=ep_train.values
        ))

        # 1. Ordinary Least Squares (OLS)
        if 'ols' in models:
            ols = LinearRegression()
            ols.fit(X_train, y_train)
            yhat_ols = ols.predict(X_test)
            eval_and_report(y_test, yhat_ols, "OLS")

            # Print top coefficients
            ols_coef = pd.Series(ols.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top OLS coeffs:\n", ols_coef.head(10))

            results[contract].update({
                "ols_model": ols,
                "ols_coefs": ols_coef,
                "train_data_ols": pd.Series(ols.predict(X_train), index=y_train.index, name=f"ols_train"),
                "yhat_ols": pd.Series(yhat_ols, index=y_test.index, name=f"{contract}_{target}_ols_pred"),
            })

        # 2. Ridge with CV over alphas (time-series CV)
        if 'ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
            ridge.fit(X_train, y_train)
            yhat_ridge = ridge.predict(X_test)
            eval_and_report(y_test, yhat_ridge, f"Ridge (alpha={ridge.alpha_:.4g})")

            # Print top coefficients
            ridge_coef = pd.Series(ridge.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Ridge coeffs:\n", ridge_coef.head(10))
            
            results[contract].update({
                "ridge_model": ridge,
                "ridge_coefs": ridge_coef,
                "train_data_ridge": pd.Series(ridge.predict(X_train), index=y_train.index, name=f"ridge_train"),
                "yhat_ridge": pd.Series(yhat_ridge, index=y_test.index, name=f"{contract}_{target}_ridge_pred"),
            })

        # 3. Lasso with CV over alphas (time-series CV)
        if 'lasso' in models and not 'lasso_ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            lasso = LassoCV(alphas=alphas, cv=tscv, fit_intercept=True, n_jobs=-1)
            lasso.fit(X_train, y_train)
            yhat_lasso = lasso.predict(X_test)
            eval_and_report(y_test, yhat_lasso, f"Lasso (alpha={lasso.alpha_:.4g})")

            # Print top coefficients
            lasso_coef = pd.Series(lasso.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso coeffs:\n", lasso_coef.head(10))

            results[contract].update({
                "lasso_model": lasso,
                "lasso_coefs": lasso_coef,
                "train_data_lasso": pd.Series(lasso.predict(X_train), index=y_train.index, name=f"lasso_train"),
                "yhat_lasso": pd.Series(yhat_lasso, index=y_test.index, name=f"{contract}_{target}_lasso_pred"),
            })

        # 4. ElasticNet with CV over alphas and l1_ratios (time-series CV)
        if 'enet' in models:
            alphas = np.logspace(-4, 6, 100)
            l1_ratios = np.arange(.1, 1, .1)   # 1.0 == Lasso, 0.0 == Ridge

            enet = ElasticNetCV(
                alphas=alphas,
                l1_ratio=l1_ratios,
                cv=tscv,
                fit_intercept=True,
                max_iter=20000,
                n_jobs = -1
            )
            enet.fit(X_train, y_train)

            yhat_enet = enet.predict(X_test)
            eval_and_report(y_test, yhat_enet, f"ElasticNet (alpha={enet.alpha_:.4g}, l1_ratio={enet.l1_ratio_})")

            # Print top coefficients
            enet_coef = pd.Series(enet.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top ElasticNet coeffs:\n", enet_coef.head(10))

            results[contract].update({
                "enet_model": enet,
                "enet_coegs": enet_coef,
                "train_data_enet": pd.Series(enet.predict(X_train), index=y_train.index, name=f"enet_train"),
                "yhat_enet": pd.Series(yhat_enet, index=y_test.index, name=f"{contract}_{target}_enet_pred"),
            })

        # 5. PCA on OLS
        if 'pca' in models:
            pca = PCA(n_components=.9).fit(X_train) # keep x% of variance and fit to training set
            train_pca = pca.transform(X_train)
            test_pca = pca.transform(X_test)

            ols_pca = LinearRegression()
            ols_pca.fit(train_pca, y_train)
            yhat_pca = ols_pca.predict(test_pca)
            eval_and_report(y_test, yhat_pca, "OLS+PCA")

            results[contract].update({
                "pca_model": ols_pca,
                "train_data_pca": pd.Series(ols_pca.predict(train_pca), index=y_train.index, name=f"pca_train"),
                "yhat_pca": pd.Series(yhat_pca, index=y_test.index, name=f"{contract}_{target}_pca_pred")
            })

        # 6. Use lasso into ridge
        if 'lasso_ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            lasso = LassoCV(alphas=alphas, cv=tscv, fit_intercept=True)
            lasso.fit(X_train, y_train)
            yhat_lasso = lasso.predict(X_test)
            eval_and_report(y_test, yhat_lasso, f"Lasso (alpha={lasso.alpha_:.4g})")

            # Print top coefficients
            lasso_coef = pd.Series(lasso.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso coeffs:\n", lasso_coef.head(10))

            results[contract].update({
                "lasso_model": lasso,
                "lasso_coefs": lasso_coef,
                "train_data_lasso": pd.Series(lasso.predict(X_train), index=y_train.index, name=f"lasso_train"),
                "yhat_lasso": pd.Series(yhat_lasso, index=y_test.index, name=f"{contract}_{target}_lasso_pred"),
            })
        
            # Get columns where coefficients are nonzero
            important_features = X_train.columns[np.abs(lasso.coef_) > 0]
            X_train_reduced = X_train[important_features]
            X_test_reduced = X_test[important_features]

            alphas = np.logspace(-4, 3, 30)

            lasso_ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
            lasso_ridge.fit(X_train_reduced, y_train)
            yhat_lasso_ridge = lasso_ridge.predict(X_test_reduced)
            eval_and_report(y_test, yhat_lasso_ridge, f"Lasso_Ridge (alpha={lasso_ridge.alpha_:.4g})")

            # Print top coefficients
            lasso_ridge_coef = pd.Series(lasso_ridge.coef_, index=important_features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso_Ridge coeffs:\n", lasso_ridge_coef.head(10))

            results[contract].update({
                "lasso_ridge_model": lasso_ridge,
                "lasso_ridge_coefs": lasso_ridge_coef,
                "train_data_lasso_ridge": pd.Series(lasso_ridge.predict(X_train_reduced), index=y_train.index, name=f"lasso_ridge_train"),
                "yhat_lasso_ridge": pd.Series(yhat_lasso_ridge, index=y_test.index, name=f"{contract}_{target}_lasso_ridge_pred"),
            })

        # 7. Partial Least Squares (PLS) Regression
        if 'pls' in models:
            # First, find the best number of components using cross-validation
            best_n = -1
            best_score = -np.inf
            
            # Test a range of components (e.g., from 1 to 20)
            for n in range(1, 8):
                pls_cv = PLSRegression(n_components=n)
                # Use a time-series friendly CV
                scores = cross_val_score(pls_cv, X_train, y_train, cv=tscv, scoring='r2')
                
                if np.mean(scores) > best_score:
                    best_score = np.mean(scores)
                    best_n = n

            print(f"PLS best n_components: {best_n}")

            # Now, fit the final PLS model with the optimal number of components
            pls = PLSRegression(n_components=best_n)
            pls.fit(X_train, y_train)
            yhat_pls = pls.predict(X_test)
            eval_and_report(y_test, yhat_pls, f"PLS (n={best_n})")

            # can get coefficients if needed, but they are in PLS component space
            # pls_coef = pd.Series(pls.coef_, index=features)

            # Store results
            results[contract].update({
                "pls_model": pls,
                # "pls_coefs": pls_coef,
                "train_data_pls": pd.Series(pls.predict(X_train).ravel(), index=y_train.index, name=f"pls_train"),
                "yhat_pls": pd.Series(yhat_pls.ravel(), index=y_test.index, name=f"{contract}_{target}_pls_pred"),
            })
            


    return results

In [ ]:
drift_results = get_or_build_results('model_data/drift_training_results.pkl.gz', train_models, drift_train_scaled, drift_test_scaled, linear_model_target, models=linear_models)

Make into new df

In [ ]:
def df_from_predictions(data, target, results, contracts=contracts, models=linear_models):
    df = data.copy()

    for contract in contracts:
        for model in models:
            df[f'{contract}_{target}_pred_{model}'] = pd.concat([results[contract][f'train_data_{model}'],results[contract][f'yhat_{model}']])
    df.dropna(inplace=True)

    return df


In [ ]:
drift = df_from_predictions(drift, linear_model_target, drift_results)

In [ ]:
drift

Plot these results

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def plot_results(data, baseline, measures, title, subtitles, contracts=contracts, models=linear_models):
    fig = make_subplots(
        rows=len(contracts), cols=1, shared_xaxes=True, vertical_spacing=0.06,
        subplot_titles=[contract for contract in contracts]
    )

    for i, (contract) in enumerate(contracts, start=1):
        show_leg = (i == 1)

        # True next_ret
        fig.add_trace(
            go.Scatter(x=data.index, y=data[f'{contract}_{baseline}'], name=f'{contract}_{baseline}',
                    mode="lines", line=dict(width=1.6),
                    showlegend=show_leg, legendgroup="true"),
            row=i, col=1
        )

        # Plot predictions
        for model in models:
            for measure in measures:
                fig.add_trace(
                    go.Scatter(x=data.index, y=data[f'{contract}_{measure}_{model}'], name=f"{model} prediction",
                            mode="lines", line=dict(width=1.4, dash="dot"),
                            showlegend=show_leg, legendgroup=f"{model}"),
                    row=i, col=1
                )

    fig.update_layout(
        title=title,
        height=2500,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(t=80, r=30, b=80, l=70),
        legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
    )

    for r in range(1, len(contracts) + 1):
        fig.update_yaxes(title_text=subtitles, row=r, col=1)

    fig.show()

In [ ]:
plot_results(drift, 'log_close_ret', [f'{linear_model_target}_pred'], 'Linear Regression on Drift', 'Log Return')

Overall returns

In [ ]:
def build_price_path(df, pred, linear_model_target=linear_model_target, contracts=contracts, models=linear_models):
    df = df.copy()

    for contract in contracts:
        for model in models:
            df[f'{contract}_{pred}_pred_{model}'] = np.exp(df[f'{contract}_log_close'].iloc[0]) * np.exp(df[f'{contract}_{linear_model_target}_pred_{model}'].cumsum())

            df[f'{contract}_close'] = np.exp(df[f'{contract}_log_close'].iloc[0]) * np.exp(df[f'{contract}_log_close_ret'].cumsum())

    return df

In [ ]:
plot_drift = build_price_path(drift, 'price')
plot_results(plot_drift, 'close', ['price_pred'], 'Price Path Evolution of Drift', 'Price')

# Volatility Modeling

In [ ]:
vol_data = data.copy()
vol_data


In [ ]:
# Squared residuals from previous -- using ElasticNet
# Turns out we actually need to also take the log because they are so tiny

linear_drift_model = 'enet'
volatility_model_target = 'log_squared_residual'

for contract in contracts:
    vol_data[f'{contract}_{volatility_model_target}'] = np.log(np.square(drift[f'{contract}_{linear_model_target}'] - drift[f'{contract}_{linear_model_target}_pred_{linear_drift_model}']))
vol_data.dropna(inplace=True)

In [ ]:
vol_training_data, vol_testing_data = split_data(vol_data)
vol_targets, vol_all_features, vol_unscale_features, vol_scale_features = set_targets_features(volatility_model_target, vol_training_data)


# Random forest for now because it's faster
vol_pruned_features = get_or_build_results('model_data/vol_features.pkl.gz', determine_feature_importance, vol_training_data, vol_testing_data, vol_all_features, volatility_model_target, mode='random_forest')

for contract in contracts:
    print(vol_pruned_features[contract]['selected_features'])

In [ ]:
vol_train_scaled, vol_test_scaled = scale_features(vol_training_data, vol_testing_data, vol_scale_features)

In [ ]:
vol_test_scaled.dropna(inplace=True)

In [ ]:
volatility_models = ['ridge', 'lasso', 'pca']

In [ ]:
vol_results = get_or_build_results('model_data/vol_results.pkl.gz', train_models, vol_train_scaled, vol_test_scaled, volatility_model_target, pruned_features=vol_pruned_features, models=volatility_models)

Make into new df

In [ ]:
diffusion = df_from_predictions(vol_data, volatility_model_target, vol_results, models=volatility_models)

In [ ]:
plot_results(diffusion, volatility_model_target, [f'{volatility_model_target}_pred'], 'Linear Regression on Diffusion', 'Log Residuals', models=volatility_models)

Convert to variance / std

In [ ]:
for contract in contracts:
    for model in volatility_models:
        diffusion[f'{contract}_variance_pred_{model}'] = np.exp(diffusion[f'{contract}_{volatility_model_target}_pred_{model}'])
        diffusion[f'{contract}_std_pred_{model}'] = np.sqrt(diffusion[f'{contract}_variance_pred_{model}'])

In [ ]:
predictions = pd.merge(drift, diffusion, left_index=True, right_index=True, how='inner', suffixes=('', '_to_drop'))
predictions = predictions.drop(columns=[col for col in predictions.columns if col.endswith('_to_drop')])

Let's add a rolling IC column to test with damping mu projections later

In [ ]:
for contract in contracts:
    predictions[f'{contract}_{linear_model_target}'] = predictions[f'{contract}_log_close'].diff().shift(-1).dropna(inplace=True)
    for model in linear_models:
        predictions[f'{contract}_IC_{model}'] = predictions[f'{contract}_{linear_model_target}'].rolling(window=21).corr(predictions[f'{contract}_{linear_model_target}_pred_{model}'])

## Looks like we obtained a pretty smooth path. There are many extreme values not being captured -- Thus the motivation for GARCH

# GARCH Volatility Model

In [ ]:
from arch import arch_model

Set up some GARCH models, check their fit our data

In [ ]:
garch_data = drift.copy()
garch_scale = 100


# Get drift residuals:
for contract in contracts:
    garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] = drift[f'{contract}_{linear_model_target}'] - drift[f'{contract}_{linear_model_target}_pred_{linear_drift_model}'] # Calculate residual from enet
    garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] = garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] * garch_scale # Scaling for better convergence

# Split data
garch_training, garch_testing = split_data(garch_data)

# Models, distributions, and paramters to check

garch_models = {
    'Garch': 'Garch',
    'Egarch': 'Egarch',
    'GJR_Garch': 'Garch',
    'APARCH': 'APARCH'
}

dists = ['normal', 't', 'skewt']

params = {
    'Garch': {'p': 1, 'o': 0, 'q': 1},
    'Egarch': {'p': 2, 'o': 1, 'q': 1},
    'GJR_Garch': {'p': 1, 'o': 1, 'q': 1},
    'APARCH': {'p': 1, 'o': 1, 'q': 1}
}

Grid search for rigorous model fitting

In [ ]:
from itertools import product

In [ ]:
def qlike(y2, fvar):
    y2 = np.asarray(y2, float)
    fvar = np.asarray(fvar, float)
    return np.mean(y2 / fvar + np.log(fvar))

def fit_one(y, vol_name, model_name, p, o, q, dist, mean='Constant', lags=0):
    kw = dict(mean=mean, lags=lags, vol=vol_name, dist=dist, p=p, q=q)
    if model_name != 'Garch':  # only asym metrics use 'o'
        kw['o'] = o
    am = arch_model(y, **kw)
    return am.fit(disp='off')

def fit_arx(y, X, vol_name, model_name, p, o, q, dist, mean, lags):
    kw = dict(mean=mean, lags=lags, x=X, vol=vol_name, dist=dist, p=p, q=q)
    if model_name != 'Garch':  # only asym metrics use 'o'
        kw['o'] = o
    am = arch_model(y, **kw)
    return am.fit(disp='off')

def rolling_oos_qlike(y, vol_name, model_name, p, o, q, dist,
                      mean='Constant', lags=0, refit_every=21, start_frac=0.6):
    """1-step-ahead variance forecasts with periodic refits."""
    y = y.dropna()
    n = len(y)
    start = int(n * start_frac)
    fvars = []
    res = None
    for t in range(start, n - 1):
        if res is None or ((t - start) % refit_every == 0):
            res = fit_one(y.iloc[:t + 1], vol_name, model_name, p, o, q, dist, mean, lags)
        fvar = res.forecast(horizon=1, reindex=False).variance.values[-1, 0]
        fvars.append(fvar)
    return qlike((y**2).iloc[start+1:], fvars)

def garch_grid_search(
    garch_training,             # DataFrame with your series, e.g., f'{contract}_drift_residuals_enet'             # dict like {'Garch':'GARCH','Egarch':'EGARCH','GJR_Garch':'GJR-GARCH','APARCH':'APARCH'}
    dists=('t','skewt'),
    P=(1,2,3), Q=(1,2,3), O=(0,1),
    means=('Constant', 'Zero'), lags=(0,1,2),
    contracts=contracts,
    garch_models = garch_models,
    compute_oos=False, refit_every=21, start_frac=0.6, arx=False, X=None
):
    rows = []
    for contract in contracts:
        y = garch_training[f'{contract}_drift_residuals_{linear_drift_model}'].dropna()
        for model_name, vol_name in garch_models.items():
            # Build parameter grid; for plain GARCH, force o=0 only
            grid_O = (0,) if model_name == 'Garch' else O
            for p, q, o, dist, lag, mean in product(P, Q, grid_O, dists, lags, means):
                try:
                    if arx:
                        res = fit_arx(y, X, vol_name, model_name, p, o, q, dist, mean, lag)
                    else:
                        res = fit_one(y, vol_name, model_name, p, o, q, dist, mean, lag)
                    aic = res.aic
                    bic = res.bic
                    llf = res.loglikelihood
                    row = {
                        'contract': contract,
                        'model': model_name,
                        'vol': vol_name,
                        'dist': dist,
                        'p': p, 'o': o, 'q': q,
                        'aic': aic, 'bic': bic, 'llf': llf,
                        'lags': lag,
                        'mean': mean
                    }
                    if compute_oos:
                        try:
                            oos = rolling_oos_qlike(y, vol_name, model_name, p, o, q, dist,
                                                    mean, lags, refit_every, start_frac)
                        except Exception:
                            oos = np.nan
                        row['oos_qlike'] = oos
                    rows.append(row)
                except Exception:
                    # skip failed fits
                    print('Failed fit, skipping')
                    continue
    out = pd.DataFrame(rows)
    if out.empty:
        return out
    sort_cols = ['oos_qlike','aic'] if compute_oos else ['aic']
    ascending = [True] * len(sort_cols)
    return out.sort_values(sort_cols, ascending=ascending).reset_index(drop=True)

In [ ]:
#results = get_or_build_results('model_data/garch_grid_search', garch_grid_search, garch_training)


In [ ]:
#results.drop_duplicates(subset=['contract'])

In [ ]:
garch_test_models = ['Egarch', 'APARCH']

## Forecast our testing period with the chosen models

In [ ]:
refit_freq = 21 # Frequency to refit at -- slightly larger helps with signal:noislambda, but not too big

def forecast_garch(garch_data, contracts=contracts, models=garch_test_models, linear_drift_model=linear_drift_model):
    garch_training, garch_testing = split_data(garch_data)

    for contract in contracts:
        for model in models:
            drift_residuals = garch_data[f'{contract}_drift_residuals_{linear_drift_model}']

            scaled_predictions = []
            for i in range(len(garch_training), len(garch_training) + len(garch_testing)): # Walk forward, forecast each step and refit model based on new inputs

                # Ensure we fit on first day and at our frequency
                testing_day = i - len(garch_training)
                is_refit_day = testing_day % refit_freq == 0
                
                if is_refit_day:
                    garch = arch_model(drift_residuals.iloc[:i], vol=garch_models[model], **params[model], mean='Zero', dist='skewt')
                    garch_fit = garch.fit(disp='off') # fit

                forecast = garch_fit.forecast(horizon=1)

                pred_variance = forecast.variance.iloc[-1].values[0]
                scaled_predictions.append(pred_variance)

            # Get exponentiation of std
            if model == 'APARCH':
                delta = garch_fit.params['delta']
            else:
                delta = 2
            
            unscaled_predictions = pd.Series(scaled_predictions, index=garch_testing.index) / (garch_scale ** delta) # Unscale (inputs scaled by 100 for convergence, var is squared, but APARCH may return delta =/= 2)
            final_predictions = unscaled_predictions ** (1 / delta)

            garch_testing[f'{contract}_std_pred_{model}'] = final_predictions

    return garch_testing.to_dict()

In [ ]:
garch_testing = pd.DataFrame(get_or_build_results('model_data/garch_testing_df.pkl.gz', forecast_garch, garch_data))
garch_testing

In [ ]:
for contract in contracts:
    for model in garch_test_models:
        predictions[f'{contract}_std_pred_{model}'] = garch_testing[f'{contract}_std_pred_{model}']

# Backtesting

In [ ]:
from scipy.stats import norm # For confidence scaling testing

Sizing strategy 1: Using Merton portfolio optimization

In [ ]:
def kelly_fraction(mu, rf, sigma, fraction=.5):
    size = fraction * (mu - rf) / (sigma**2 + 1e-8) 
    return size

Sizing strategy 2: Using vol target

In [ ]:
def get_vol_target_sizing(target, vol):
    size = (target/vol)
    return size

Returns info

In [ ]:
def sharpe_and_mdd(df, col_equity, col_rf="ten_yr", periods_per_year=252):
    eq = df[col_equity]
    rf = df[col_rf]

    # compute portfolio simple returns
    port_rets = eq.pct_change().dropna()
    # align rf
    rf_aligned = rf.reindex(port_rets.index).astype(float)

    # excess returns (assuming rf is already per-period, e.g. daily)
    excess = port_rets - ((1+rf_aligned/100)**(1/periods_per_year)-1)
    sharpe = (excess.mean() / excess.std()) * np.sqrt(periods_per_year) if excess.std() > 0 else np.nan

    # max drawdown
    running_max = eq.cummax()
    drawdown = eq / running_max - 1.0
    max_dd = drawdown.min()

    return sharpe, max_dd

## Trading strategy 1: Trade based on our models

### Backtest

In [ ]:
bt_models = ['ridge', 'lasso', 'APARCH', 'Egarch'] # Volatility models

Get historical margin amounts

In [ ]:
historical_margins = {}

for contract in contracts:
    historical_margins[contract] = pd.read_csv(f'{contract}_Historical_Margin.csv')
    historical_margins[contract]['Date'] = pd.to_datetime(historical_margins[contract]['Date'])
    historical_margins[contract].set_index(historical_margins[contract]['Date'], inplace=True)
    historical_margins[contract].drop(columns=['Date'], inplace=True)
    historical_margins[contract].rename(columns={'Long Margin': f'{contract}_Long', 'Short Margin': f'{contract}_Short'}, inplace=True)

### Individual Contract BTs

Backtesting with margin functionality (with rules per CME future challenge)  
Note that I am omitting the minimum 10 per day contract trading rule because we can just buy and sell quickly to nullify the effect  
I am also omitting the liquidation before expiry rule because there are no expiries during the competition  

It might be slower, but using iterrows is simpler

In [ ]:
# Set initial conditions
starting_cash = 500_000
commission = 2.5
drawdown_lock = .2

initial_margin = 1.1 # 10% extra assumption on initial entry
tick_size = {
    'ES': 0.25, 
    'NQ': 0.25, 
    'FV': 0.0078125, 
    'TY': 0.015625, 
    'US': 0.03125, 
    'GC': 0.10, 
    'CL': 0.01, 
    'JY': 0.0000005,
    'BP': 0.0001, 
    'EU': 0.00005
}

tick_value = {
    'ES': 12.50, 
    'NQ': 5.00, 
    'FV': 7.8125, 
    'TY': 15.625, 
    'US': 31.25, 
    'GC': 10.00, 
    'CL': 10.00, 
    'JY': 6.25,
    'BP': 6.25, 
    'EU': 6.25
}

In [ ]:
# Create backtest df copy
bt = predictions.copy()
bt = bt.join([margin_data for margin_data in historical_margins.values()])
bt = bt.replace(0, pd.NA)
bt = bt.fillna(method='ffill')
bt_train, bt_test = split_data(bt)
linear_drift_model = 'enet'

bt_data = {}

for contract in contracts:
    bt_data[contract] = {}

    for model in bt_models:
        bt_test[f'{contract}_cash_{model}'] = 0.0
        bt_test[f'{contract}_margin_{model}'] = 0.0
        bt_test.loc[bt_test.index[0], f'{contract}_cash_{model}'] = starting_cash
        bt_test[f'{contract}_signal_{model}'] = 0
        bt_test[f'{contract}_baseline_wealth'] = np.exp(bt_test[f'{contract}_log_close_ret'].cumsum()) * starting_cash
        bt_test[f'{contract}_margin_usage_{model}'] = 0


        bt_data[contract][model] = {
            'last_margin': 0,
            'last_cash': starting_cash,
            'position': 0,
            'average_vol_weight': 0,
            'average_confidence': 0,
            'average_vol': 0,
            'average_mu': 0,
            'average_rf': 0,
            'average_ret': 0,
            'high_water': 0,
        }

# Starts at second day
for idx, row in bt_test.iloc[1:].iterrows():
    for contract in contracts:
        vol_data = []
        for model in bt_models:
            last_margin = bt_data[contract][model]['last_margin']
            last_cash = bt_data[contract][model]['last_cash']

            multiplier = tick_value[contract] / tick_size[contract]
            position_size = bt_data[contract][model]['position']
            lock_up = False

            
            vol = row[f'{contract}_std_pred_{model}']

            # Mu needs to be adjusted upwards for sigma, notice no exponentiation is needed since we are using continuous time from log gbm assumption
            if model not in garch_models.keys():
                mu = np.exp(row[f'{contract}_{linear_model_target}_pred_{linear_drift_model}'] + .5 * vol**2) - 1
            else:
                # We need a different adjustment for our garch models since they are not based on a normality assumption
                garch_drift_adjustment = + (1/2) * vol**2 - 1/6 * vol**3
                mu = np.exp(row[f'{contract}_{linear_model_target}_pred_{linear_drift_model}'] + garch_drift_adjustment) - 1
                
            rf = (1+row['^TNX_close']/100)**(1/252)-1 # Daily
            investment = 0
            ret = np.exp(row[f'{contract}_log_close_ret']) - 1
            delta = ret*np.exp(row[f'{contract}_log_open']) # This calculation is basically assuming we are making our trading decisions at the end of every day
            pred = mu*np.exp(row[f'{contract}_log_close'])
            

            # Update margin value
            pnl_today = bt_data[contract][model]['position'] * delta * multiplier

            # Calculate total equity
            last_margin += pnl_today
            total_equity = last_margin + last_cash

            # Calculate max portfolio value for drawdown constraints
            bt_data[contract][model]['high_water'] = max(total_equity, bt_data[contract][model]['high_water'])

            # Check maintenance requirements
            if position_size > 0:
                direction = 'Long'
                side = 1
            else:
                direction = 'Short'
                side = -1

            margin_difference = row[f'{contract}_{direction}']*position_size - last_margin
            if margin_difference > 0:
                # If enough cash to top up, do it automatically
                if total_equity >= margin_difference:
                    last_cash -= margin_difference
                    last_margin += margin_difference
                else:
                    # Otherwise liquidate
                    bt_data[contract][model]['position'] = 0
                    last_cash += last_margin # Ignoring commission
                    last_margin = 0
            else:
                last_cash += margin_difference
                last_margin -= margin_difference

            # Recalculate total equity
            total_equity = last_margin + last_cash

            # If drawdown criteria met, lock portfolio for day
            if ret <= -drawdown_lock:
                lock_up = True

            if not lock_up:
                # Get bet sizing with fractional kelly -- default is .5
                max_leverage = 5
                target_notional = kelly_fraction(mu, rf, vol, fraction=1) * total_equity # Daily rf
                target_notional = max(-total_equity * max_leverage, min(target_notional, total_equity * max_leverage)) # Make sure we don't exceed equity

                # Get bet sizing with fixed vol target
                #base_weight = get_vol_target_sizing(.15 / np.sqrt(252), vol)
                #confidence = (mu - rf) / vol
                #k=2
                #target_weight = base_weight * norm.cdf(confidence*np.sqrt(252))*k # Annualize
                #target_notional = target_weight * total_equity
                #target_notional = max(-total_equity, min(target_notional, total_equity)) # Make sure we don't exceed equity / effective equity cap

                # Data logging
                bt_data[contract][model]['average_rf'] += rf
                bt_data[contract][model]['average_ret'] += ret
                bt_data[contract][model]['average_mu'] += mu
                bt_data[contract][model]['average_vol'] += vol
                #bt_data[contract][model]['average_vol_weight'] += base_weight
                #bt_data[contract][model]['average_confidence'] += norm.cdf(confidence*np.sqrt(252))*k

                # Execute trade / Update investment amount to reach target position
                if target_notional > 0:
                    direction = 'Long'
                else: 
                    direction = 'Short'

                notional_per_contract = np.exp(row[f'{contract}_log_close']) * multiplier
                initial_cost = row[f'{contract}_{direction}']*initial_margin # Get current maintenance * initial margin percentage requirement
                position = math.floor(target_notional/notional_per_contract) # Get number of contracts to calculate margin investment, ignoring commission
                investment = position*initial_cost - last_margin
                investment = max(-last_cash, min(investment, last_cash))

                bt_data[contract][model]['position'] = position

            else:
                investment = 0

            # Update cash
            last_cash = bt_data[contract][model]['last_cash'] = last_cash - investment
            bt_test.loc[idx, f'{contract}_cash_{model}'] = last_cash

            # Store portfolio value as last value for calculation
            bt_data[contract][model]['last_margin'] = last_margin + investment
            bt_test.loc[idx, f'{contract}_margin_{model}'] = bt_data[contract][model]['last_margin']
            bt_test.loc[idx, f'{contract}_margin_usage_{model}'] = last_margin/total_equity

            # Set signal for plotting later
            if investment > 0:
                bt_test.loc[idx, f'{contract}_signal_{model}'] = 1
            elif investment < 0:
                bt_test.loc[idx, f'{contract}_signal_{model}'] = -1
    
for contract in contracts:
    print(f'=== {contract} ===')
    total_value = f'{contract}_baseline_wealth'
    
    final_training_value = bt_test[total_value].iloc[-1]
    training_cagr = 100 * ((final_training_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {contract}, baseline: {final_training_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {training_cagr:.2f}%\n")
    for model in bt_models:
        
        total_value = f'{contract}_total_value_{model}'
        bt_test[total_value] = bt_test[f'{contract}_margin_{model}'] + bt_test[f'{contract}_cash_{model}']

        final_testing_value = bt_test[total_value].iloc[-1]
        testing_cagr = 100*((final_testing_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
        print(f'Final value for {contract}, {model}: {final_testing_value:.2f}')
        
        test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
        print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {testing_cagr:.2f}%")


        bt_data[contract][model]['average_mu'] /= len(bt_test)
        bt_data[contract][model]['average_vol'] /= len(bt_test)
        bt_data[contract][model]['average_rf'] /= len(bt_test)
        bt_data[contract][model]['average_ret'] /= len(bt_test)
        average_margin_util = bt_test[f'{contract}_margin_usage_{model}'].sum() / len(bt_test)
        print(f"Average Vol: {bt_data[contract][model]['average_vol']:.4f}, Average mu: {bt_data[contract][model]['average_mu']:.4f}, Average ret: {bt_data[contract][model]['average_ret']:.4f}")
        print(f"Average Margin Utilizization: {average_margin_util*100:.2f}%")
        print('\n')

In [ ]:
plot_results(bt_test, 'baseline_wealth', ['total_value'], 'Backtest Performance', 'Wealth', models=bt_models, contracts=contracts)

### Let's try making a single portfolio now 

In [ ]:
def plot_bt(data, baseline, measures, title, subtitles, models=bt_models):
    fig = make_subplots(
        rows=len(models), cols=1, shared_xaxes=True, vertical_spacing=0.06,
        subplot_titles=[model for model in models]
    )

    for i, (model) in enumerate(models, start=1):
        show_leg = (i == 1)

        # True next_ret
        fig.add_trace(
            go.Scatter(x=data.index, y=data[f'{baseline}_{model}'], name=f'{baseline}_{model}',
                    mode="lines", line=dict(width=1.6),
                    showlegend=show_leg, legendgroup="true"),
            row=i, col=1
        )

        # Plot predictions
        for measure in measures:
            fig.add_trace(
                go.Scatter(x=data.index, y=data[f'{measure}_{model}'], name=f"{model} prediction",
                        mode="lines", line=dict(width=1.4, dash="dot"),
                        showlegend=show_leg, legendgroup=f"{measure}"),
                row=i, col=1
            )

    fig.update_layout(
        title=title,
        height=1200,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(t=80, r=30, b=80, l=70),
        legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
    )

    for r in range(1, len(contracts) + 1):
        fig.update_yaxes(title_text=subtitles, row=r, col=1)

    fig.show()

Historical Correlation

In [ ]:
# Calculate historical correlation matrix for train returns
contract_corr = bt_train[[f'{contract}_log_close_ret' for contract in contracts]].corr()


In [ ]:
def multi_asset_kelly_fraction(mu, sigma, fraction=.5):
    weights = fraction * np.linalg.solve(sigma, mu) 
    return weights

In [ ]:
from tqdm import tqdm

Backtesting on multiple assets

In [ ]:
def multi_asset_backtest(max_leverage=15, kelly_frac=.5, bt_models=bt_models):
    # Create backtest df copy
    bt = predictions.copy()
    bt = bt.join([margin_data for margin_data in historical_margins.values()])
    bt = bt.replace(0, pd.NA)
    bt = bt.fillna(method='ffill')
    bt_train, bt_test = split_data(bt)
    linear_drift_model = 'enet'

    multipliers = np.array([tick_value[contract]/tick_size[contract] for contract in contracts])

    T = len(bt_test)

    close_cols  = [f'{contract}_log_close' for contract in contracts]
    ret_cols   = [f'{contract}_log_close_ret' for contract in contracts]
    long_cols  = [f'{contract}_Long' for contract in contracts]
    short_cols = [f'{contract}_Short' for contract in contracts]
    mu_cols    = [f'{contract}_{linear_model_target}_pred_{linear_drift_model}' for contract in contracts]

    rf_mat = np.power((1+(bt_test['^TNX_close'].to_numpy(float))/100), 1/252)-1  # Daily
    raw_mu_mat = bt_test[mu_cols].to_numpy(float)
    ret_mat = np.exp(bt_test[ret_cols].to_numpy(float)) - 1
    close_mat = np.exp(bt_test[close_cols].to_numpy(float))
    maintL_mat = bt_test[long_cols].to_numpy(float)
    maintS_mat = bt_test[short_cols].to_numpy(float)

    for model in bt_models:
        bt_test[f'cash_{model}'] = 0.0
        bt_test[f'cash_{model}'].iloc[0] = starting_cash
        bt_test[f'margin_{model}'] = 0.0
        equal_weighted_returns = np.exp(bt_test[ret_cols].cumsum())
        portfolio_values = equal_weighted_returns * (starting_cash / len(contracts))
        bt_test[f'baseline_wealth_{model}'] = portfolio_values.sum(axis=1)
        bt_test[f'margin_usage_{model}'] = 0
        
        vol_cols = [f'{contract}_std_pred_{model}' for contract in contracts]
        pred_vol_mat = bt_test[vol_cols].to_numpy(float)
        pred_var_mat = np.square(pred_vol_mat)

        margins = np.full(len(contracts),0.0)
        positions = np.full(len(contracts),0)
        last_total_equity = 0
        total_equity = 0
        cash = starting_cash

        # Loop
        for t in tqdm(range(1, T)):
            rf = rf_mat[t]
            lock_up = False

            # Get predicted variances for next day and create covariance matrix based on historical correlation
            pred_vol = pred_vol_mat[t]
            D = np.diag(pred_vol)
            pred_cov = D @ contract_corr @ D

            # mu needs to be adjusted upwards for sigma
            raw_mu = raw_mu_mat[t]
            if model not in garch_models.keys():
                mu = np.exp(raw_mu + (1/2) * pred_var_mat[t]) - 1
            else:
                # We need a different adjustment for our garch models since they are not based on a normality assumption
                garch_drift_adjustment = (1/2) * pred_var_mat[t] - (1/6) * np.power(pred_vol_mat[t], 3)
                mu = np.exp(raw_mu + garch_drift_adjustment) - 1

            # Update margin value
            prevClose = close_mat[t-1]
            close = close_mat[t]
            ret = ret_mat[t]
            delta = ret*prevClose # This calculation is basically assuming we are making our trading decisions at the end of every day
            pnl_today = positions * delta * multipliers
            margins += pnl_today

            # Calculate total equity for lockup criteria
            last_total_equity = total_equity
            total_equity = margins.sum() + cash

            # If drawdown criteria met, lock portfolio for day
            lock_up = total_equity/(last_total_equity + 1e-8) <= (1 - drawdown_lock)

            # Check maintenance requirements
            maint_long = maintL_mat[t]
            maint_short = maintS_mat[t]
            maint_side = np.where(positions > 0, maint_long, maint_short)

            maint_diff = maint_side*positions - margins

            if maint_diff.sum() > cash:
                # Liquidate -- simple, not greedy (may be less effective)
                cash += margins.sum()
                margins *= 0
                positions *= 0
            else:
                # Top ups & off load to cash -- notice that 'negative' maint_diff means return to normal maintenance and add to cash
                margins += maint_diff
                cash -= maint_diff.sum()

            # Recalculate total equity
            total_equity = margins.sum() + cash

            if not lock_up:
                # Get bet sizing with fractional kelly -- default is .5
                target_notionals = np.clip(multi_asset_kelly_fraction(mu-rf, pred_cov, fraction=kelly_frac), -max_leverage, max_leverage) * total_equity
                maint_side = np.where(target_notionals > 0, maint_long, maint_short)

                # Portfolio allocation
                position_found = False
                k = 1 # Scaling constant to reduce portfolio investment if needed
                while not position_found and k > 0:
                    notional_per_contract = close * multipliers
                    initial_costs = maint_side*initial_margin # Get current maintenance * initial margin percentage requirement
                    target_positions = target_notionals * k / notional_per_contract # Get number of contracts to calculate margin investment, ignoring commission
                    target_positions = np.where(target_positions > 0, np.floor(target_positions), np.ceil(target_positions))
                    
                    investments = target_positions*initial_costs - margins
                    
                    required_margin = np.abs(investments).sum()
                    total_investment = investments.sum()

                    if required_margin.sum() > cash:
                        k -= .05
                    else:
                        position_found = True
            else:
                investments = np.full(len(contracts),0.0)
                total_investment = 0
                target_positions = positions

            # Update cash
            cash -= total_investment
            bt_test.iloc[t, bt_test.columns.get_loc(f'cash_{model}')] = cash

            # Update positions and margins
            positions = target_positions
            margins += investments
            bt_test.iloc[t, bt_test.columns.get_loc(f'margin_{model}')] = margins.sum()
        
        # Update models in bt_test
        total_value = f'total_value_{model}'
        bt_test[total_value] = bt_test[f'margin_{model}'] + bt_test[f'cash_{model}']

    return bt_test

In [ ]:
bt_test = multi_asset_backtest(max_leverage=16.5, kelly_frac=.9)

In [ ]:
for model in bt_models:
    print(f'=== {model} ===')
    total_value = f'baseline_wealth_{model}'
    
    final_training_value = bt_test[total_value].iloc[-1]
    training_cagr = 100 * ((final_training_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {contract}, baseline: {final_training_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {training_cagr:.2f}%\n")

    total_value = f'total_value_{model}'

    final_testing_value = bt_test[total_value].iloc[-1]
    testing_cagr = 100*((final_testing_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {model}: {final_testing_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {testing_cagr:.2f}%")

    average_margin_util = bt_test[f'margin_usage_{model}'].sum() / len(bt_test)
    print(f"Average Margin Utilizization: {average_margin_util*100:.2f}%")
    print('\n')

### Hyperparameter grid search

In [ ]:
import itertools

In [ ]:
# Define the parameter ranges
leverages = np.arange(10.0, 20.0, 0.5)  # A more reasonable range than 1-50
kelly_fracs = np.arange(0.1, 1.1, 0.1)
results = {}
testing_model = 'APARCH'

# Create the grid of all parameter combinations
param_grid = list(itertools.product(leverages, kelly_fracs))
print(f"Starting grid search for {len(param_grid)} parameter combinations...")

for leverage, kelly_frac in tqdm(param_grid):
    # The dictionary key is now a tuple of the parameters
    param_key = (leverage, kelly_frac)
    try:
        bt_test = multi_asset_backtest(max_leverage=leverage, kelly_frac=kelly_frac, bt_models=[testing_model])

        total_value_col = f'total_value_{testing_model}'
        final_value = bt_test[total_value_col].iloc[-1]
        sharpe, mdd = sharpe_and_mdd(bt_test, total_value_col, col_rf="^TNX_close")

        results[param_key] = (final_value, sharpe)
    except Exception as e:
        print(f"Backtest failed for params {param_key}. Reason: {e}")
        results[param_key] = (np.nan, np.nan)

# --- Analysis: Find the Best Result ---

# Filter out failed runs
valid_results = {k: v for k, v in results.items() if isinstance(v, tuple) and not np.isnan(v[1])}

if valid_results:
    # Find the parameter tuple (key) that corresponds to the maximum Sharpe ratio (the 2nd element of the value)
    best_params = max(valid_results, key=lambda k: valid_results[k][1])
    
    # Unpack the parameters and the results
    best_leverage, best_kelly_frac = best_params
    best_total_value, best_sharpe_ratio = valid_results[best_params]

    print("\n--- Optimization Complete (based on Sharpe Ratio) ---")
    print(f"Best Sharpe Ratio: {best_sharpe_ratio:.4f}")
    print(f"Total Value at Best Sharpe: ${best_total_value:,.2f}")
    print(f"Achieved with Max Leverage: {best_leverage:.2f}")
    print(f"Achieved with Kelly Fraction: {best_kelly_frac:.2f}")

else:
    print("\nAll backtests failed.")

# --- Better Analysis with Pandas ---
print("\n--- Top 10 Results ---")
# Convert results to a DataFrame for easier analysis
records = []
for params, metrics in valid_results.items():
    records.append({
        'Leverage': params[0],
        'KellyFrac': params[1],
        'FinalValue': metrics[0],
        'SharpeRatio': metrics[1]
    })
results_df = pd.DataFrame(records)
print(results_df.sort_values(by='SharpeRatio', ascending=False).head(10).round(4))

In [ ]:
plot_bt(bt_test, 'baseline_wealth', ['total_value'], 'Backtest Performance', 'Wealth', models=bt_models)